In [15]:
import os
from tqdm import tqdm
from datasets import load_dataset
import sys
sys.path.append('../evaluation')
from evaluate import RetrievalSystem, main as evaluate_main
import yaml
import hyde

In [4]:
with open("/users/christineye/retrieval/config.yaml", 'r') as stream:
    api_key = yaml.safe_load(stream)['openai_api_key']

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
test_hyde = hyde.HydeRetrievalSystem(config_path = "/users/christineye/retrieval/config.yaml", 
                                     db_path = "", generate_n = 3, embed_query = True, max_doclen = 100)

In [27]:
test_hyde.retrieve("What are the types of Type 1a supernovae?")

['Abstract:\nType Ia supernovae (SNe Ia) are a class of stellar explosions that play a crucial role in our understanding of cosmology and the chemical evolution of galaxies. These events are characterized by the thermonuclear disruption of a white dwarf star, which can occur through various mechanisms. In this paper, we review the current understanding of the different types of SNe Ia, their progenitor systems, and the implications for our knowledge of stellar evolution and the universe.\n\nConclusion:\nOur analysis of the different types of SNe Ia reveals the complexity and diversity of these events. The single-degenerate and double-degenerate scenarios, as well as the sub-Chandrasekhar and super-Chandrasekhar mass progenitors, all contribute to the observed population of SNe Ia. Understanding these various pathways is crucial for refining our models of stellar evolution, nucleosynthesis, and the role of SNe Ia in shaping the chemical and physical properties of galaxies. Continued obs

IndexError: list index out of range